In [1]:
import cv2
import numpy as np
from mediapipe.python.solutions import face_mesh
import math
import joblib
import winsound  


def draw_axes(img, yaw, pitch, roll, tdx=None, tdy=None, size=80):
    # bn7awel el zawayaa men degrees le radians 3shan nesta3mel trig functions
    pitch = pitch * np.pi / 180
    yaw = -yaw * np.pi / 180  # el "-" 3shan el direction bet3 yaw mo3akes
    roll = roll * np.pi / 180

    # lw m3telnash el center point (tdx, tdy), ben7aded nos el sora
    if tdx is None or tdy is None:
        height, width = img.shape[:2]  # bengeb el height w width mn el sora
        tdx = width // 2               # nos el width
        tdy = height // 2              # nos el height

    # ======================= X Axis (Red) =======================
    # bn7seb el makan el gdyd le el x-axis ba3d el lfa (yaw, pitch, roll)
    x1 = int(tdx + size * (math.cos(yaw) * math.cos(roll)))
    y1 = int(tdy + size * (math.cos(pitch) * math.sin(roll) + math.cos(roll) * math.sin(pitch) * math.sin(yaw)))
    # bnrsom el x-axis line bel ahmar
    cv2.line(img, (tdx, tdy), (x1, y1), (0, 0, 255), 2)

    # ======================= Y Axis (Green) =======================
    # bn7seb makan el y-axis ba3d el lfa
    x2 = int(tdx + size * (-math.cos(yaw) * math.sin(roll)))
    y2 = int(tdy + size * (math.cos(pitch) * math.cos(roll) - math.sin(pitch) * math.sin(yaw) * math.sin(roll)))
    # bnrsom el y-axis line bel akhdar
    cv2.line(img, (tdx, tdy), (x2, y2), (0, 255, 0), 2)

    # ======================= Z Axis (Blue) =======================
    # bn7seb makan el z-axis ba3d el lfa
    x3 = int(tdx + size * (math.sin(yaw)))
    y3 = int(tdy + size * (-math.cos(yaw) * math.sin(pitch)))
    # bnrsom el z-axis line bel azra2
    cv2.line(img, (tdx, tdy), (x3, y3), (255, 0, 0), 2)


# Load the trained models
model_yaw = joblib.load(r"D:\senior-2\semester 2\Machine Vision\svm_yaw_model2.pkl")["model"]
model_pitch = joblib.load(r"D:\senior-2\semester 2\Machine Vision\svm_pitch_model2.pkl")["model"]
model_roll = joblib.load(r"D:\senior-2\semester 2\Machine Vision\svm_roll_model2.pkl")["model"]

cap = cv2.VideoCapture(0)

with face_mesh.FaceMesh(static_image_mode=False) as mesh:
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image horizontally
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face in results.multi_face_landmarks:
                h, w = frame.shape[:2]
                x = [int(lm.x * w) for lm in face.landmark]
                y = [int(lm.y * h) for lm in face.landmark]

                # Normalize as in training
                x_centered = np.array(x) - x[99]
                y_centered = np.array(y) - y[99]
                face_width = np.linalg.norm([x[10]-x[171], y[10]-y[171]])
                feature = np.hstack([x_centered, y_centered]) / face_width
                feature = feature.reshape(1, -1)

                # Predict angles
                yaw = model_yaw.predict(feature)[0]
                pitch = model_pitch.predict(feature)[0]
                roll = model_roll.predict(feature)[0]

                # Get nose tip coordinates
                nose_tip = face.landmark[1]  
                tdx = int(nose_tip.x * w)
                tdy = int(nose_tip.y * h)

                draw_axes(frame, yaw, pitch, roll, tdx, tdy)

                # Display angles on screen
                text = f"Yaw: {yaw:.2f}, Pitch: {pitch:.2f}, Roll: {roll:.2f}"
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                            0.8, (255, 255, 255), 2, cv2.LINE_AA)
                # Check for excessive pitch or roll
                if abs(pitch) > 0.5 or abs(roll) > 0.5:
                    cv2.putText(frame, "WARNING: sleeping driver!", (10, 70), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 0, 255), 3, cv2.LINE_AA)
                    winsound.Beep(1000, 200)  # Play a warning beep

        cv2.imshow("Head Pose", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [4]:
pip install scikit-learn==1.6.1


Note: you may need to restart the kernel to use updated packages.
